# **Set Analyzer**

Le but de ce script est d'analyser statistiquement les différents set de carte du jeu *Magic the Gathering* afin de :


1.   Déterminer les cartes les plus pertinentes dans chaque set
2.   Comparer les métadonnées liées aux cartes entre chaque set

L'objet de l'analyse est de pouvoir objectiver la valeur intrinsèque d'une carte *intra* et *inter* set.

L'application visée est l'utilisation de ses données afin de performer dans les formats limités (*sealed, draft*)




***draft***

quelques métadonnées utiles

**intraset**
- nombre de cartes + ratios
- card type : couleur, créature / non-créature, rareté
- cout : valeur de mana
- board : power, toughness, evasion (en fonction de keywords: flying, trample, menace, ...)
- interaction : (en fonction de keywords: counter, return, deal damage, ...)
- mana fixing : type de manafix (en fonction de keywords: dork, rock, treasures, ...)

**interset**
- replacer chacune des données précédentes par rapport aux précédents sets
- évaluer : vitesse du set, créatures fragiles, propension de bombes, propension d'interaction, potentiel de fixing, etc





# **Initialisation**

In [4]:
# Import librairies to use in the code

import os
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm

# Import widgets librairies
# import ipywidgets as widgets
from IPython.display import display

Upload data from JSON (dataset from https://mtgjson.com/)

In [6]:
# Set path of the folder containing dataset
dataset_FolderPath = Path.cwd().parent / 'datasets' / 'MTG_datasets' # @dev TBC before each use

# Set path of the File
dataset_FileName = 'AllPrintings.json'
dataset_FilePath = dataset_FolderPath / dataset_FileName

In [7]:
# load all dataset
data = pd.read_json(dataset_FilePath)
allSets = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

In [8]:
setCompare = allSets.apply(pd.Series)[['baseSetSize', 'code', 'totalSetSize', 'type', 'name', 'releaseDate']]

## Load Limited dataset function

In [10]:
def loadLimitedSet(set_code):
    """
    Loads and processes a subset of Magic: The Gathering cards from a given set code.
    
    Parameters:
    set_code (str): The unique identifier of the card set to be loaded.
    
    Returns:
    pd.DataFrame: A DataFrame containing the cleaned and filtered card data.
    
    Functionality:
    1. Defines relevant card features to be analyzed, including name, keywords, mana cost, color identity, 
       power, toughness, rarity, types, and text.
    2. Loads the card data from the `allSets` dataset based on the given `_setCode`.
    3. Converts the extracted data into a Pandas DataFrame and selects only the first `baseSetSize` cards.
    4. Cleans numeric fields ('manaValue', 'power', 'toughness') by converting them to numeric values, 
       coercing invalid entries to NaN.
    5. Filters the dataset to include only cards with rarities 'common' or 'uncommon'.
    6. Returns the processed DataFrame for further analysis.
    """
    
    # Define cards features to be analyzed
    features_analyzed = [
        'name',
        'keywords',
        'manaValue',
        'manaCost',
        'colorIdentity',
        'power',
        'toughness',
        'rarity',
        'types',
        'text']

    # Load cards
    df = pd.DataFrame.from_dict(allSets.loc[set_code]['cards'])
    cards = df.loc[:allSets.loc[set_code]['baseSetSize']-1-5, features_analyzed] # remove the last 5 cards that are always basic lands
    
    # Clean numeric data
    cards[['manaValue', 'power', 'toughness']] = cards[['manaValue', 'power', 'toughness']].apply(pd.to_numeric, errors='coerce')
    
    # Keep only common and uncommon cards
    cards = cards[cards['rarity'].isin(['common', 'uncommon'])]
    
    return cards

In [43]:
set_code = 'OTJ'
cards = loadLimitedSet(set_code)
cards

,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
0,Breaker of Creation,"[Annihilator, Hexproof, Hexproof from]",8.0,{6}{C}{C},[],8.0,4.0,uncommon,[Creature],"When you cast this spell, you gain 1 life for ..."
2,Drownyard Lurker,"[Cycling, Vigilance]",7.0,{7},[U],7.0,7.0,common,[Creature],Vigilance\nWhen you cast or cycle Drownyard Lu...
4,Eldrazi Ravager,"[Annihilator, Cycling]",6.0,{5}{C},[],6.0,6.0,uncommon,[Creature],"Annihilator 1 (Whenever this creature attacks,..."
6,Glaring Fleshraker,NaN,3.0,{2}{C},[],2.0,2.0,uncommon,[Creature],"Whenever you cast a colorless spell, create a ..."
8,It That Heralds the End,NaN,2.0,{1}{C},[],2.0,2.0,uncommon,[Creature],Colorless spells you cast with mana value 7 or...
...,...,...,...,...,...,...,...,...,...,...
290,Waterlogged Teachings // Inundated Archive,NaN,0.0,NaN,"[B, U]",NaN,NaN,uncommon,[Land],Inundated Archive enters tapped.\n{T}: Add {U}...
291,Angel of the Ruins,"[Cycling, Flying, Plainscycling]",7.0,{5}{W}{W},[W],5.0,7.0,uncommon,"[Artifact, Creature]","Flying\nWhen Angel of the Ruins enters, exile ..."
292,Decree of Justice,[Cycling],4.0,{X}{X}{2}{W}{W},[W],NaN,NaN,uncommon,[Sorcery],Create X 4/4 white Angel creature tokens with ...
293,Distinguished Conjurer,NaN,2.0,{1}{W},[W],1.0,2.0,uncommon,[Creature],"Whenever another creature you control enters, ..."


## 1) SPEED

Format speed can be caracterized by :
- the ratio of creatures: set a grade from 100-90% == S ; 90-70% == A ; 70-50% == B ; 50-30% == C ; 30-10% == D ; 10-0% == E
- the median creature `manaValue`
- the median `powerToManaValue` : above 1: creatures hit hard, fast
- the board state (see section 2)
- the number of interactions (see section 3)

In [46]:
# Filter for 'Creature' only
cardsCreatureFiltered = cards[cards['types'].apply(lambda x: 'Creature' in x)]
cardsCreatureFiltered = cardsCreatureFiltered.copy()

# Ratio of creatures
nTot = len(cards)
nCreature = len(cardsCreatureFiltered)
limitedCreatureRatio = (nCreature / nTot) * 100 # in percentage
# add a grade (@dev TBD)

# Creature Manavalue
meanCreatureMV = cardsCreatureFiltered['manaValue'].mean()
cardsCreatureFiltered['normalizedCreatureManaValue'] = cardsCreatureFiltered['manaValue'] - meanCreatureMV # normalized columns

# Creature Power to ManaValue
cardsCreatureFiltered['powerToManaValue'] = cardsCreatureFiltered['power'] / cardsCreatureFiltered['manaValue']
meanPowerToMV = cardsCreatureFiltered['powerToManaValue'].mean()
cardsCreatureFiltered['normalizedPowerToManaValue'] = cardsCreatureFiltered['power'] - meanPowerToMV # normalized columns

In [48]:
# Add values to setCompare
setCompare = setCompare.copy()

setCompare.at[set_code, 'limited_CreatureRatio'] = limitedCreatureRatio
setCompare.at[set_code, 'limited_meanCreatureManaValue'] = meanCreatureMV
setCompare.at[set_code, 'limited_meanCreaturePowerToManaValue'] = meanPowerToMV

## 2) BOARD STATE

- the mean creature `power`
- the mean creature `thougness`
- the mean `powerToToughness ratio`: above 1: creatures are likely to hit harder and defend badly (and vice versa)
- ratio of evasive creatures (ie. 'Flying', 'Trample', 'Menace')

In [75]:
# Creature Power
meanCreaturePower = cardsCreatureFiltered['power'].mean()
cardsCreatureFiltered['normalizedCreaturePower'] = cardsCreatureFiltered['power'] - meanCreaturePower # normalized columns

# Creature Toughness
meanCreatureToughness = cardsCreatureFiltered['toughness'].mean()
cardsCreatureFiltered['normalizedCreatureToughness'] = cardsCreatureFiltered['toughness'] - meanCreatureToughness # normalized columns

# Creature Power to Toughness
cardsCreatureFiltered['powerToToughness'] = cardsCreatureFiltered['power'] / cardsCreatureFiltered['toughness']
meanPowerToToughness = cardsCreatureFiltered['powerToToughness'].mean()
cardsCreatureFiltered['normalizedPowerToToughness'] = cardsCreatureFiltered['power'] - meanPowerToToughness # normalized columns

# Evasion
def countKeywords(data):
    unique_keywords = list(data['keywords'].explode().unique())
    unique_keywords.remove(np.nan)
    exploded_data = data.explode('keywords')
    filtered_data = exploded_data[exploded_data['keywords'].isin(unique_keywords)]
    KW_count = exploded_data['keywords'].value_counts().to_dict()
    return KW_count

def countEvasiveKeywords(keyword_dict, keyword_list):
    evasiveCount = [keyword_dict[key] for key in keyword_list]
    return evasiveCount

KWCount = countKeywords(cardsCreatureFiltered)

#evasiveKW = ['Flying', 'Trample', 'Menace']
#evasiveKWCount = countEvasiveKeywords(KWCount, evasiveKW)

In [77]:
# Add values to setCompare
setCompare = setCompare.copy()
setCompare.at[set_code, 'limited_meanCreaturePower'] = meanCreaturePower
setCompare.at[set_code, 'limited_meanCreatureToughness'] = meanCreatureToughness
setCompare.at[set_code, 'limited_meanCreaturePowerToToughness'] = meanPowerToToughness
setCompare.at[set_code, 'limited_KWCount'] = [KWCount]
#setCompare.at[set_code, 'limited_evasiveKWCount'] = [evasiveKWCount]

ValueError: Must have equal len keys and value when setting with an ndarray

## 3) FIXING

- monocolor-to-multicolor ratio (lands excluded)
- multi-pip ratio : cards with more that one colored pip in mana cost
- ratio of mana producer + types (lands, manadorks, manarocks, treasures)
- type of mana produced (TBD)

In [56]:
# Monocolor to multicolor ratio
non_land_cards_total = len(cards[(cards['types'].apply(lambda x: 'Land' not in x))])
multicolor_nonland_cards = len(
    cards[
        (cards['types'].apply(lambda x: 'Land' not in x)) 
        & (cards['colorIdentity'].apply(len) > 1)
    ])
monocolorToMulticolorRatio = multicolor_nonland_cards / non_land_cards_total

# Multi-pip ratio
def isMultiPip(s, letters_to_remove=None):
    if letters_to_remove is None:
        letters_to_remove = ['{', '}', 'C', 'X']  # Assign default list safely

    if not isinstance(s, str):  # Handle NaN or non-string values
        return False
    
    s = ''.join(c for c in s if c not in letters_to_remove and not c.isdigit())
    return len(s) > 1

multiPipRatio = len(cards[cards['manaCost'].apply(isMultiPip)]) / non_land_cards_total

# Mana producers
def producesMana(s): #@dev TO BE TESTED FOR FETCH
    pattern1 = r'add (?:\d+|one|two|three|four|five) mana'
    match1 = re.search(pattern1, s, re.IGNORECASE)

    pattern2 = r'add {'
    match2 = re.search(pattern2, s, re.IGNORECASE)
    
    return bool(match1 or match2)

# Ratio of mana producers
n_manaProducer = len(cards[cards['text'].apply(producesMana)])
n_nonLand_manaProducer = len(
    cards[
        cards['text'].apply(producesMana) 
        & (cards['types'].apply(lambda x: 'Land' not in x))
    ])
manaProducerRatio = n_manaProducer / nTot
nonLand_manaProducerRatio = n_nonLand_manaProducer / nTot

# Type of producer
def countManaProducerTypes(data):
    df = data[data['text'].apply(producesMana)]

    # Non-basic Lands
    a = len(df[
            df['types'].apply(lambda x: 'Land' in str(x))
            ]) 
    # Dorks (that do not produces treasures)
    b = len(df[
            df['types'].apply(lambda x: 'Creature' in str(x)) 
            & df['keywords'].apply(lambda x: 'Treasure' not in str(x))
            ]) 
    # Rocks (artifacts that are not creatures and do not produce treasures)
    c = len(df[
            df['types'].apply(lambda x: 'Artifact' in str(x)) 
            & df['types'].apply(lambda x: 'Creature' not in str(x)) 
            & df['keywords'].apply(lambda x: 'Treasure' not in str(x))
            ]) 
    # Treasures
    d = len(df[
            df['keywords'].apply(lambda x: 'Treasure' in str(x))
            ])

    # @dev, here does not account for any other type of mana production (ie. Dark Ritual)

    producer_type = {
        'Lands': a,
        'Dorks': b,
        'Rocks': c,
        'Treasures': d
    }
    return producer_type

manaProducerTypes = countManaProducerTypes(cards)

# Type of mana produced
# @dev, TBD in the future

In [57]:
setCompare.at[set_code, 'limited_MonoToMulticolorRatio'] = monocolorToMulticolorRatio
setCompare.at[set_code, 'limited_MultiPipRatio'] = multiPipRatio
setCompare.at[set_code, 'limited_manaProducerRatio'] = manaProducerRatio
setCompare.at[set_code, 'limited_nonLand_manaProducerRatio'] = nonLand_manaProducerRatio
setCompare.at[set_code, 'limited_manaProducerTypes'] = [manaProducerTypes]

## 4) Interactions (TBD)

a quel point le set est interactif ?
000 - définir ce qu'est une interaction
- ratio de permanents
- pourcentage d'interaction
- la "vitesse" de l'interaction = distribution de mana value des sorts interactifs
- type d'interaction : single-target removal + combat trick
- color pie

In [60]:
"""
# ratio of permanents

def checklist(items_wanted, items_tbc):
  return any(item in items_wanted for item in items_tbc)

permanent_index = [item for item in type_index if (item !='Instant' and item!='Sorcery')]
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==False)] #non-permanent
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)]  #permanents

print('permanent ratio = ' + str(len(cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)])/len(cards)*100) + ' %')
"""

"\n# ratio of permanents\n\ndef checklist(items_wanted, items_tbc):\n  return any(item in items_wanted for item in items_tbc)\n\npermanent_index = [item for item in type_index if (item !='Instant' and item!='Sorcery')]\ncards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==False)] #non-permanent\ncards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)]  #permanents\n\nprint('permanent ratio = ' + str(len(cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)])/len(cards)*100) + ' %')\n"

In [62]:
"""
# get interactive cards

interaction_list = [
    'destroy',
    'exile',
    'counter',
    'target'
]

def interactive_card(str):
  if any(word in str for word in interaction_list):
    return True
  else:
    return False

cards[cards['text'].apply(interactive_card)]
"""

"\n# get interactive cards\n\ninteraction_list = [\n    'destroy',\n    'exile',\n    'counter',\n    'target'\n]\n\ndef interactive_card(str):\n  if any(word in str for word in interaction_list):\n    return True\n  else:\n    return False\n\ncards[cards['text'].apply(interactive_card)]\n"

# **Interset**

In [66]:
# toutes les stats interset

# mettre en input le nombre et la temporalité des sets à comparer (tout le modern, 4 derniers sets, etc)
# écrire une ligne d'input

# appeler les fonctions précédentes
# ranger dans des listes / df pour faire les statistiques ensuite

In [73]:
setCompare.loc[['OTJ', 'MH3']]

,baseSetSize,code,totalSetSize,type,name,releaseDate,limited_CreatureRatio,limited_medianCreatureManaValue,limited_medianCreaturePowerToManaValue,limited_medianCreaturePower,limited_medianCreatureToughness,limited_medianCreaturePowerToToughness,limited_KWCount,limited_MonoToMulticolorRatio,limited_MultiPipRatio,limited_manaProducerRatio,limited_nonLand_manaProducerRatio,limited_manaProducerTypes
OTJ,286,OTJ,374,expansion,Outlaws of Thunder Junction,2024-04-19,52.736318,3.103774,0.826774,2.509434,2.783019,1.030680,"[{'Plot': 17, 'Flying': 16, 'Saddle': 10, 'Rea...",0.113636,0.181818,0.184080,0.059701,"[{'Lands': 25, 'Dorks': 4, 'Rocks': 1, 'Treasu..."
MH3,303,MH3,560,draft_innovation,Modern Horizons 3,2024-06-14,45.410628,3.404255,0.758589,2.537634,2.677419,1.083492,"[{'Devoid': 17, 'Flying': 14, 'Adapt': 9, 'Bes...",0.182857,0.308571,0.256039,0.101449,"[{'Lands': 32, 'Dorks': 14, 'Rocks': 2, 'Treas..."
